- Ggf. zu float16 konvertieren checken, ob finaler df mit time slices dann deutlich kleiner und performance testen
- Cross validation?
- Ensemble learning?

In [1]:
#Import data handling libraries
import pandas as pd
import numpy as np

In [2]:
#Specify directories
#code_env = 'kaggle'
code_env = 'local'


if code_env=='local':
    ###local###
    #get parent folder of current directory
    parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

    #Directory resources
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'
    prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

if code_env=='kaggle':
    ###On Kaggle###
    res_dir = '/kaggle/input/m5-forecasting-accuracy/'
    prc_dir = '/kaggle/input/processed-data/'

In [3]:
# Create variables
BASE     = prc_dir +'df_1.pkl'
CALENDAR = prc_dir +'df_2.pkl'

In [4]:
# Read in df_train_conv from pickle file
df_all_data = pd.concat([pd.read_pickle(BASE),
           pd.read_pickle(CALENDAR)], 
           axis=1)

In [5]:
# Splitting the data
TRAIN_END = 1913 - 28 # 1885 -> Train only until the 28 days before the end of the data
VAL_END = 1913

df_train = df_all_data[df_all_data['d'] <= TRAIN_END]
#df_val = df_all_data[(df_all_data['d'] > TRAIN_END) & (df_all_data['d'] <= VAL_END)]
#df_test = df_all_data[df_all_data['d'] > VAL_END]

del df_all_data

In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam
from keras import backend as K

In [7]:
# Define categorical and numerical columns
categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 
                    'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
                    'snap_CA', 'snap_TX', 'snap_WI', 'mday', 'week', 'month', 'year']
numerical_cols = ['sell_price']

target_col = 'sales_amount'

# Convert categorical columns to category dtype and encode with cat.codes
for col in categorical_cols:
    df_train[col] = df_train[col].astype('category').cat.codes

# Normalize numerical columns
scaler = MinMaxScaler()
df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols].astype(np.float32))
df_train['sales_amount'] = scaler.fit_transform(df_train[['sales_amount']].astype(np.float32))


# LSTM Data Preparation
time_steps = 7  # Number of time steps (can be tuned)
batch_size = 32  # Batch size for the generator


In [8]:
# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [9]:
# Custom Generator Function
def lstm_data_generator(df, target, days_per_sequence=1, batch_size=32):
    num_items = df[df['d'] == 1].shape[0]  # 30490 -> Number of items per day
    total_sequences = (len(df) - num_items * days_per_sequence) // num_items # 1878
    while True:  # Loop indefinitely, the model's fit method will break the loop
        for i in range(0, total_sequences, batch_size):
            batch_sequences = []
            batch_targets = []
            for b in range(batch_size):
                if i + b < total_sequences: # 0, 0; 0, 1; 0, 2; ...; 0, 32; 32, 0; 32, 1; ...
                    start_idx = (i + b) * num_items
                    end_idx = start_idx + num_items * days_per_sequence
                    batch_sequences.append(df.iloc[start_idx:end_idx, :].drop(target, axis=1).to_numpy()) #drop target column, Only the values in the DataFrame will be returned, the axes labels will be removed.
                    batch_targets.append(df.iloc[end_idx:end_idx + num_items][target].to_numpy())
            yield np.array(batch_sequences), np.array(batch_targets)

# Usage
time_steps = 7  # Number of days per sequence
batch_size = 32  # Size of each batch
train_gen = lstm_data_generator(df_train, target_col, days_per_sequence=time_steps, batch_size=batch_size)
num_items = df_train[df_train['d'] == 1].shape[0]  # Number of items per day (day 1 examplary)


In [10]:
#train_gen.__next__()[1].shape  # (32, 30490, 16) -> (batch_size, num_items, num_features)

In [11]:
# Define the LSTM Model
input_shape = (time_steps * num_items, len(df_train.columns) - 1)  # Adjust input shape accordingly
model_input = Input(shape=input_shape)
x = LSTM(50, activation='relu')(model_input)
x = Dense(1)(x)
model = Model(inputs=model_input, outputs=x)

model.compile(optimizer=Adam(), loss=rmse)


In [12]:
# Train the model using the generator
model.fit(train_gen, steps_per_epoch=(len(df_train) // (batch_size * num_items)), epochs=10)

Epoch 1/10


2024-01-09 09:12:23.766743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-09 09:12:23.788249: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 7/58 [==>...........................] - ETA: 32:15 - loss: nan

In [ ]:
# def create_sequences(df, target, days_per_sequence=1):    
#     num_items = df_train[df_train['d'] == 1].shape[0]  # Number of items per day (day 1 examplary)
#     X, y = [], []

#     for i in range(0, len(df) - num_items * days_per_sequence, num_items):
#         end_idx = i + num_items * days_per_sequence
#         X.append(df.iloc[i:end_idx, :].drop(target, axis=1).values)
#         y.append(df.iloc[end_idx:end_idx + num_items][target].values)

#     return np.array(X), np.array(y)

# # Usage
# time_steps = 7  # Number of days per sequence
# X_train, y_train = create_sequences(df_train, target_col, days_per_sequence=time_steps)


In [ ]:
# Model

In [ ]:
# After prediction
# predictions_normalized = model.predict(...)  # Predictions in the normalized scale
# predictions_original = scaler.inverse_transform(predictions_normalized)  # Inverse-transformed predictions